In [4]:

import time
import pathlib
import sys
sys.path.insert(0, str(pathlib.Path("..").absolute()))

from computation_sim.nodes import (
    ConstantNormalizer,
    FilteringMISONode,
    Node,
    OutputNode,
    PeriodicEpochSensor,
    RingBufferNode,
    SinkNode,
    SourceNode,
)
from computation_sim.system import Action, System, SystemDrawer
from computation_sim.time import Clock, FixedDuration
from IPython.display import display, clear_output

In [5]:
clock = Clock(0)

# Normalizer for time
normalizer = ConstantNormalizer(100.0)

# Setup the source
sensor = PeriodicEpochSensor(0, 100, FixedDuration(0))
source_node_0 = SourceNode(clock.as_readonly(), sensor, "SOURCE_NODE_0")
sensor = PeriodicEpochSensor(0, 100, FixedDuration(0))
source_node_1 = SourceNode(clock.as_readonly(), sensor, "SOURCE_NODE_1")

# Setup sink and output
output_node = OutputNode(clock.as_readonly(), id="OUTPUT_NODE", age_normalizer=normalizer)
lost_node_0 = SinkNode(clock.as_readonly(), id="LOST_MESSAGES_0")
lost_node_1 = SinkNode(clock.as_readonly(), id="LOST_MESSAGES_1")

# Setup buffers
buffer_0 = RingBufferNode(
    clock.as_readonly(),
    "BUFFER_0",
    age_normalizer=normalizer,
)
buffer_1 = RingBufferNode(
    clock.as_readonly(),
    "BUFFER_1",
    age_normalizer=normalizer,
)

# Setup the processor node
compute_node = FilteringMISONode(
    clock.as_readonly(),
    FixedDuration(50),
    id="COMPUTE_NODE",
    age_normalizer=normalizer,
)

# Connect outputs
source_node_0.add_output(buffer_0)
source_node_1.add_output(buffer_1)
buffer_0.set_output(compute_node)
buffer_1.set_output(compute_node)
buffer_0.set_overflow_output(lost_node_0)
buffer_1.set_overflow_output(lost_node_0)
compute_node.set_output_fail(lost_node_1)
compute_node.set_output_pass(output_node)

# Set-up the action
compute_action = Action(name="ACT_COMPUTE_NODE")
compute_action.register_callback(buffer_0.trigger, 1)
compute_action.register_callback(buffer_1.trigger, 1)
compute_action.register_callback(compute_node.trigger, 0)

system = System()
system.add_action(compute_action)
system.add_node(lost_node_0)
system.add_node(lost_node_1)
system.add_node(output_node)
system.add_node(compute_node)
system.add_node(source_node_0)
system.add_node(source_node_1)
system.add_node(buffer_0)
system.add_node(buffer_1)

In [7]:

clock.reset()
system.reset()
system.update()
drawer = SystemDrawer()
drawer.build(system._node_graph)
clear_output(wait=True)
display(drawer.fw)
    
while clock.get_time() < 10000:
    print(clock.get_time())
    system.update()
    if (clock.get_time() + 10) % 100 == 0:
        system.act([1])
    drawer.update(system._node_graph)
    clear_output(wait=True)
    display(drawer.fw)
    clock += 10
    time.sleep(0.1)


        

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': '#888', 'width': 0.5},
              'mode': 'lines',
              'showlegend': False,
              'type': 'scatter',
              'uid': 'e511b35e-b4bd-4246-9730-bfd80bc93bcb',
              'x': [0.3333333333333333, 1.0, None, 0.3333333333333333, 1.0, None,
                    -1.0, -0.3333333333333333, None, -0.3333333333333333,
                    0.3333333333333333, None, -0.3333333333333333,
                    0.3333333333333333, None, -1.0, -0.3333333333333333, None,
                    -0.3333333333333333, 0.3333333333333333, None,
                    -0.3333333333333333, 0.3333333333333333, None],
              'y': [-0.3333333333333333, -0.3333333333333333, None,
                    -0.3333333333333333, 0.3333333333333333, None,
                    -0.3333333333333333, -0.3333333333333333, None,
                    -0.3333333333333333, -0.3333333333333333, None,
                    -0.3333

KeyboardInterrupt: 